## LeNet5 实现Mnist手写数字识别

![image.png](./img/lenet5.png)

In [8]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data

IMAGE_SIZE = 28
NUM_CHANNELS = 1
CONV1_SIZE = 5
CONV1_KERNEL_NUM = 32
CONV2_SIZE = 5
CONV2_KERNEL_NUM = 64
FC_SIZE = 512
OUTPUT_NODE = 10

In [9]:
def get_weight(shape, regularizer, name='weights'):
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1), name = name)
    if regularizer:
        tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape,name = 'bias'):
    return tf.Variable(tf.zeros(shape), name=name)

def conv2d(x,w,name='conv'):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME',name=name)

def max_pool_2x2(x,name='max_pool'):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME', name=name)

In [10]:
def forward(x,train,regularizer):
    #                    [5,5,1,32]
    conv1_w = get_weight([CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_KERNEL_NUM],regularizer=0.0001, name='conv1_w')
    conv1_b = get_bias([CONV1_KERNEL_NUM],name='conv1_b')
    #       [28,28,32]
    conv1 = conv2d(x,conv1_w, name='conv1')
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_b))
    #       [14,14,32]
    pool1= max_pool_2x2(relu1,name='pool1')
    
    #                   [5,5,32,64]
    conv2_w = get_weight([CONV2_SIZE,CONV2_SIZE,CONV1_KERNEL_NUM,CONV2_KERNEL_NUM],regularizer=0.0001, name='conv2_w')
    conv2_b = get_bias([CONV2_KERNEL_NUM],name='conv2_b')
    #           [14,14,64]
    conv2 = conv2d(pool1,conv2_w, name='conv2')
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2,conv2_b))
    #           [7,7,64]
    pool2= max_pool_2x2(relu2,name='pool2')
    
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1] * pool_shape[2] * pool_shape[3]
    #                            [1,7*7*64]
    reshaped = tf.reshape(pool2, [pool_shape[0], nodes])
    
    fc1_w = get_weight([nodes,FC_SIZE], regularizer=0.0001, name='fc1_w')
    fc1_b = get_bias([FC_SIZE], name='fc1_b')
    fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_w)+fc1_b)
    if train:
        fc1 = tf.nn.dropout(fc1,0.5)
        
    fc2_w = get_weight([FC_SIZE,OUTPUT_NODE], regularizer=0.0001, name='fc2_w')
    fc2_b = get_bias([OUTPUT_NODE], name='fc2_b')
    y = tf.matmul(fc1,fc2_w)+fc2_b
    return y

In [11]:
BATCH_SIZE = 100
REGULARIZER = 0.001

LEARNING_RATE_BASE = 0.1 #最初学习率
LEARNING_RATE_DECAY = 0.99 #学习率衰减因子
MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减因子

MODEL_NAME = 'lenet'
MODEL_SAVE_PATH = './runs'
STEPS = 20000

def backward(mnist):
    #                             [batch,28,28,1]
    x = tf.placeholder(tf.float32,[BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS], name='input_x')
    #                             [None,10]
    y_ = tf.placeholder(tf.float32,[None, OUTPUT_NODE])
    
    y = forward(x,True,REGULARIZER)
    
    global_step = tf.Variable(0, trainable=False)
    
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.arg_max(y_,1),logits=y))
    reg_loss = loss + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,
                                          global_step,
                                          mnist.train.num_examples / BATCH_SIZE,
                                          LEARNING_RATE_DECAY,
                                          staircase=True,
                                          name='learning_rate')
    
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    ema_op = ema.apply(tf.trainable_variables()) #每次运行此句，所有待优化参数求滑动平均
    with tf.control_dependencies([train_step, ema_op]):
        train_op = tf.no_op(name='train')
        
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        
        for i in range(STEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs = np.reshape(xs, (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
            
            _, loss_val, step = sess.run([train_op, loss, global_step], feed_dict={x:reshaped_xs, y_:ys})
            
            if i%100==0:
                print('After %d training step(s), loss on all data is %g:' %(i, loss_val))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

In [12]:
import time 
TEST_INTERVAL_SECS = 5
def test_(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[mnist.test.num_examples, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
        
        y = forward(x, False, None)
        
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)
        
        correct_prediction = tf.equal(tf.arg_max(y,1), tf.arg_max(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        j=0
        while j<21:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    
                    xs = mnist.test.images
                    reshaped_xs = np.reshape(xs, (xs.shape[0], IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
                    
                    accuracy_score = sess.run(accuracy, feed_dict ={x: reshaped_xs, y_: mnist.test.labels})
                    print("After %s training step(s). test accuracy = %f"%(global_step, accuracy_score))
                else:
                    print("No checkpoint file found")
                    return
                time.sleep(TEST_INTERVAL_SECS)
                j+=1

In [14]:
def train():
    mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
    backward(mnist)

In [13]:
def test():
    mnist = input_data.read_data_sets("MNIST_data",one_hot = True)
    test_(mnist)

In [18]:
train()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
After 0 training step(s), loss on all data is 5.24048:
After 100 training step(s), loss on all data is 2.3173:
After 200 training step(s), loss on all data is 2.29189:
After 300 training step(s), loss on all data is 2.29587:
After 400 training step(s), loss on all data is 2.29929:
After 500 training step(s), loss on all data is 2.30922:
Instructions for updating:
Use standard file APIs to delete files with this prefix.
After 600 training step(s), loss on all data is 2.31332:
After 700 training step(s), loss on all data is 2.32196:
After 800 training step(s), loss on all data is 2.29415:
After 900 training step(s), loss on all data is 2.3056:
After 1000 training step(s), loss on all 

In [35]:
test()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.113500
INFO:tensorflow:Restoring parameters from ./runs/lenet-19901
After 19901 training step(s). test accuracy = 0.1